In [123]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [124]:
import sys
import os

# Get the absolute path of the src directory
SRC_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "src"))

# Add 'src' to sys.path if not already present
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

# Print updated sys.path
print("\n🔍 Updated sys.path:")
for p in sys.path:
    print(p)

# Now import
from Predict_Pipe.utils.common import read_yaml, create_directories
from Predict_Pipe.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH

print("\n🚀 Import successful!")



🔍 Updated sys.path:
c:\Users\maini\Anaconda3\envs\venv\python312.zip
c:\Users\maini\Anaconda3\envs\venv\DLLs
c:\Users\maini\Anaconda3\envs\venv\Lib
c:\Users\maini\Anaconda3\envs\venv

c:\Users\maini\Anaconda3\envs\venv\Lib\site-packages
c:\Users\maini\Anaconda3\envs\venv\Lib\site-packages\win32
c:\Users\maini\Anaconda3\envs\venv\Lib\site-packages\win32\lib
c:\Users\maini\Anaconda3\envs\venv\Lib\site-packages\Pythonwin
c:\Academics\AI and Ml\MLOPs\E2E-DataScience-Deployment\src
C:\Academics\AI and ML\MLOPs\src

🚀 Import successful!


In [125]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [126]:
import os
import urllib.request as request
from Predict_Pipe.logging import logger
import zipfile

In [127]:
## components- Data Ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    ## downloading ip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip filee into the data directory 
        Function return none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    


In [128]:
import os
os.chdir("C:/Academics/AI and ML/MLOPs/E2E-DataScience-Deployment")


In [130]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-31 13:21:53,310: INFO: common]: yaml file: config\config.yaml loaded sucessfully
[2025-03-31 13:21:53,311: INFO: common]: yaml file: params.yaml loaded sucessfully
[2025-03-31 13:21:53,313: INFO: common]: yaml file: schema.yaml loaded sucessfully
[2025-03-31 13:21:53,314: INFO: common]: created directory at: artifacts
[2025-03-31 13:21:53,315: INFO: common]: created directory at: artifacts/data_ingestion
[2025-03-31 13:21:55,169: INFO: 2523628334]: artifacts/data_ingestion/winequality-red.csv download! with following info: 
Server: nginx/1.27.4
Date: Mon, 31 Mar 2025 07:51:54 GMT
Transfer-Encoding: chunked
Connection: close




BadZipFile: File is not a zip file